In [224]:
import numpy as np
from sympy import symbols, cos, sin, Matrix, atan2, acos, sqrt, pi, Array
import sympy
import mpmath

# Helper functions

In [225]:
def A_dh(theta, d, a, alpha, type, radians = False):
    if not radians and not isinstance(theta, sympy.core.symbol.Symbol):
        theta = theta / 180 * sympy.pi
    if not radians and not isinstance(alpha, sympy.core.symbol.Symbol):
        alpha = alpha / 180 * sympy.pi
    return Matrix([
        [cos(theta), -sin(theta)*cos(alpha), sin(theta)*sin(alpha), a*cos(theta)],
        [sin(theta), cos(theta)*cos(alpha), -cos(theta)*sin(alpha), a*sin(theta)],
        [0, sin(alpha), cos(alpha), d],
        [0, 0, 0, 1]
    ])

def A_trans(x, y, z, type):
    return Matrix([
        [1, 0, 0, x],
        [0, 1, 0, y],
        [0, 0, 1, z],
        [0, 0, 0, 1]
    ])
    



# Config
 

In [226]:
theta_1, theta_2, theta_3, theta_4= symbols('theta_1 theta_2 theta_3 theta_4')

In [227]:
dh_conv = {
    1 : {'theta': theta_1, 'd': 50/1000, 'a': 0, 'alpha': 90, 'type': 'revolute'},
    2 : {'theta': theta_2, 'd': 0, 'a': 93/1000, 'alpha': 0, 'type': 'revolute'},
    3 : {'theta': theta_3, 'd': 0, 'a': 93/1000, 'alpha': 0, 'type': 'revolute'},
    4 : {'theta': theta_4, 'd': 0, 'a': 50/1000, 'alpha': 0, 'type': 'revolute'},
    5: {'x' : -15/1000, 'y' : 45/1000, 'z' : 0, 'type': 'transformation'}
}



In [228]:
np.radians(90)

np.float64(1.5707963267948966)

In [229]:
for i, a in dh_conv.items():
    print(a)

{'theta': theta_1, 'd': 0.05, 'a': 0, 'alpha': 90, 'type': 'revolute'}
{'theta': theta_2, 'd': 0, 'a': 0.093, 'alpha': 0, 'type': 'revolute'}
{'theta': theta_3, 'd': 0, 'a': 0.093, 'alpha': 0, 'type': 'revolute'}
{'theta': theta_4, 'd': 0, 'a': 0.05, 'alpha': 0, 'type': 'revolute'}
{'x': -0.015, 'y': 0.045, 'z': 0, 'type': 'transformation'}


In [230]:
A = {
    '01' : A_dh(**dh_conv[1]),
    '12' : A_dh(**dh_conv[2]),
    '23' : A_dh(**dh_conv[3]),
    '34' : A_dh(**dh_conv[4]),
    '45' : A_trans(**dh_conv[5])
}

In [231]:
T = {
    '00' : sympy.eye(4),
    '01' : sympy.simplify(A['01']),
    '02' : sympy.simplify(A['01']*A['12']),
    '03' : sympy.simplify(A['01']*A['12']*A['23']),
    '04' : sympy.simplify(A['01']*A['12']*A['23']*A['34']),
    '05' : sympy.simplify(A['01']*A['12']*A['23']*A['34']*A['45'])
    # '05' : sympy.simplify(A['45']*A['34']*A['23']*A['12']*A['01'])
}

In [249]:
def jacobian_inner(t, joint, o_n):
    joint_type = joint['type']
    z = t[:3,2]
    o = t[:3,3]
    
    if joint_type == 'revolute':
        j = z.cross(o_n - o)
        return Matrix.vstack(j, z)
    else:
        return Matrix.vstack(z, Matrix.zeros(3,1))
    

def jacobian(T_, joints):
    o_n = T_[-1][:3,3]
    J = Matrix.zeros(6, len(joints))
    for idx, joint in enumerate(joints):
        t = T_[idx]
        j = jacobian_inner(t, joint, o_n)
        
        J[:, idx] = j
        
    return sympy.simplify(J)

def jacobian_all(T_, dh_conv_):
    J = {}
    for idx, joint in enumerate(dh_conv_):
        t = list(T_.values())[:idx+1]
        joints = list(dh_conv_.values())[:idx+1]
        J[idx] = jacobian(t, joints)
    return J

In [250]:
def point_on_circle(phi: float, radius: float = 32/1000):
    centre = Array([150/1000, 0, 120/1000])
    return centre + radius * Array([0, cos(phi), sin(phi)])

In [251]:
def inv_kinematics(x,y,z,phi):
    """ """
    L1,L2,L3,L4 = 50/1000, 93/1000, 93/1000, 50/1000 # m

    theta1 = atan2(y, x)

    A = x - L4*cos(theta1)*cos(phi)
    B = y - L4*sin(theta1)*cos(phi)
    C = z - L1 - L4*sin(phi)

    theta3 = acos((A*A + B*B + C*C - L2*L2 - L3*L3)/(2*L2*L3))
    a = L3*sin(theta3)
    b = L2 + L3*cos(theta3)

    c = z - L1 - L4*sin(phi)
    r = sqrt(a*a + b*b)

    theta2 = atan2(c, sqrt(r*r - c*c)) - atan2(a,b)

    theta4 = phi - theta2 - theta3
    
    return theta1, theta2, theta3, theta4

In [262]:
inv_kinematics(*point_on_circle(0), 0)

(0.210182586662170, -0.239730216951784, 1.66990262059900, -1.43017240364722)

In [265]:

def jacobian_point(J, _theta_1, _theta_2, _theta_3, _theta_4):
    """Computes jacobian given joint angles"""
    res = sympy.lambdify((theta_1, theta_2, theta_3, theta_4), J, 'sympy')
    return res(_theta_1, _theta_2, _theta_3, _theta_4)

def jacobian_circle_angle(J, phi):
    """Computes jacobian given an angle of a circle"""
    
    point_on_circle(J, phi) #Compute coordinates of point on circle
    
    #Compute angles of joints
    _theta_1, _theta_2, _theta_3, _theta_4 = inv_kinematics(*point_on_circle(phi), phi)
    

    #Compute jacobian
    return jacobian_point(J, _theta_1, _theta_2, _theta_3, _theta_4)


In [ ]:
J_end = jacobian(list(T.values())[:4], list(dh_conv.values())[:3])
J_end

r1 = jacobian_circle_angle(J_end, 0)
r2 = jacobian_circle_angle(J_end, pi/2)
r3 = jacobian_circle_angle(J_end, pi)
r4 = jacobian_circle_angle(J_end, 3*pi / 2)

print(r1)
print(r2)
print(r3)
print(r4)

Matrix([[-0.0215680764114741, -0.0684594985497015, -0.0900555097184086], [0.101100358178785, -0.0146046930239363, -0.0192118420732605], [0, 0.103375356560303, 0.0130349642695982], [0, 0.208638471770519, 0.208638471770519], [0, -0.977992836424308, -0.977992836424308], [1, 0, 0]])
Matrix([[0, -0.0520000000000000, -0.0717831394489319], [0.150000000000000, 0, 0], [0, 0.150000000000000, 0.0591285116577036], [0, 0, 0], [0, -1, -1], [1, 0, 0]])
Matrix([[0.0424319235885260, -0.0684594985497015, -0.0342297492748508 - 0.0499397455863514*I], [0.198899641821215, 0.0146046930239363, 0.00730234651196816 + 0.010653812391755*I], [0, 0.203375356560303, 0.101687678280151 - 0.0175756082767954*I], [0, -0.208638471770519, -0.208638471770519], [0, -0.977992836424308, -0.977992836424308], [1, 0, 0]])
Matrix([[0, -0.0880000000000000, -0.0724502503296009], [0.150000000000000, 0, 0], [0, 0.150000000000000, 0.0583091864733008], [0, 0, 0], [0, -1, -1], [1, 0, 0]])


In [266]:
J_cam = jacobian(list(T.values())[:4], list(dh_conv.values())[:3])
J_cam

Matrix([
[-0.093*(cos(theta_2) + cos(theta_2 + theta_3))*sin(theta_1), (-0.093*sin(theta_2) - 0.093*sin(theta_2 + theta_3) + 6.93889390390723e-18)*cos(theta_1), (6.93889390390723e-18 - 0.093*sin(theta_2 + theta_3))*cos(theta_1)],
[ 0.093*(cos(theta_2) + cos(theta_2 + theta_3))*cos(theta_1), (-0.093*sin(theta_2) - 0.093*sin(theta_2 + theta_3) + 6.93889390390723e-18)*sin(theta_1), (6.93889390390723e-18 - 0.093*sin(theta_2 + theta_3))*sin(theta_1)],
[                                                          0,                                        0.093*cos(theta_2) + 0.093*cos(theta_2 + theta_3),                                       0.093*cos(theta_2 + theta_3)],
[                                                          0,                                                                             sin(theta_1),                                                       sin(theta_1)],
[                                                          0,                                              

In [257]:
r1 = jacobian_circle_angle(J, 0)
r2 = jacobian_circle_angle(J, pi/2)
r3 = jacobian_circle_angle(J, pi)
r4 = jacobian_circle_angle(J, 3*pi / 2)

In [261]:
r4

Matrix([
[   0, -0.088, -0.0724502503296009],
[0.15,      0,                   0],
[   0,   0.15,  0.0583091864733008],
[   0,      0,                   0],
[   0,     -1,                  -1],
[   1,      0,                   0]])